In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

f = open("output/log.txt")

vals = []
columns = []
for i in range(10):
    line = f.readline()
    columns.append(line.split(" at")[0])
    vals.append(float(line.split(" ")[-1].replace("\n","")))

df = pd.DataFrame(columns=columns, data=[vals])

for i in range(1, 160):
    df2 = pd.DataFrame(data=[[float(f.readline().split(" ")[-1].replace("\n","")) for k in range(10)]], columns=columns)
    df = df.append(df2)

df['index'] = list(range(160))
df = df.set_index('index')
df.head()
plt.figure(figsize=(20, 12))
sns.lineplot(data=df[[columns[k] for k in [5,8,9]]]).get_figure().savefig("output/stats_densities_time")
plt.close('all')
plt.figure(figsize=(20, 12))
sns.lineplot(data=df[[columns[k] for k in [0,1,2,3,4]]]).get_figure().savefig("output/stats_weights_time")
plt.close('all')


In [ ]:
from sim_vars import data_dir_path, T, ds_densities_sim_path
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4 as nc
import numpy as np

def gen_heatmap_visu(dataset, name):
    plt.figure(figsize=(20, 12))
    sns.heatmap(dataset.transpose(), robust=True).get_figure().savefig("output/" + name)

def compute_avg_matrix(data, T):
    n, p = len(data), len(data[0])
    avg_data = np.zeros((n, p))
    for i in range(n):
        for j in range(p):
            s = sum([data[i, j][t] for t in range(len(T))])
            avg_data[i, j] = s / len(T)
    return avg_data

ds_densities_sim = nc.Dataset(ds_densities_sim_path, "r")
densities_sim = np.array(ds_densities_sim["density"][:, :, :])
print("Computing averages for the year")
avgs_sim = compute_avg_matrix(densities_sim, T)
# gen_heatmap_visu(avgs_sim, "avg_densities_end")
sns.heatmap(avgs_sim.transpose())

In [ ]:
import numpy as np
from netCDF4 import Dataset
from sim_vars import main_dir_path, ds_parts_useful_path, ds_parts_sim_path
import matplotlib.pyplot as plt

NB_PARTS = 50000
NB_ITER = 200

ds_parts_path = (
    main_dir_path + "parts_gen_vortex_" + str(NB_PARTS) + "_" + str(NB_ITER) + ".nc"
)

ds = Dataset(ds_parts_useful_path, 'r+')

distance_to_center = lambda x,y: np.sqrt((x - 60) ** 2 + (y - 40) ** 2)

distance_to_center(ds["lon"][0][0], ds["lat"][0][0])

plt.plot(ds["lon"][4502,:], ds['lat'][4502,:])
ds["lon"][4502,2], ds['lat'][4502,2]

In [ ]:
import numpy as np
import netCDF4 as nc
from sim_vars import ds_parts_ensembles_path, ds_densities_ensembles_path, NB_ENSEMBLES

ds_parts_useful = nc.Dataset(ds_parts_ensembles_path)
ds_densities_ensembles = nc.Dataset(ds_densities_ensembles_path)
a = ds_parts_useful["weight"][:]
a[:, 0]
ds_densities_ensembles['density'][:,:,:,:].max()

In [ ]:
import pandas as pd
import numpy as np

a = np.arange(4)
a = a.reshape((2,2,1))
a[1,1,0]

In [ ]:
import netCDF4 as nc
import seaborn as sns
import numpy as np
import pandas as pd
from sim_vars import ds_densities_ref_path, main_dir_path, LONGITUDES, LATITUDES
import matplotlib.pyplot as plt

ds_densities_ref = nc.Dataset(ds_densities_ref_path)
ds_densities_ref_2 = nc.Dataset(main_dir_path + "data_double_gyre_ens_30_rad_10_devinit_0.5_obscov_10_iterstart_0_iterend_1950_nbobspoint_1//densities_double_gyre_ens_30_rad_10_devinit_0.5_obscov_10_iterstart_0_iterend_1950_nbobspoint_1_ref.nc")

np.std(ds_densities_ref_2["density"][:,:,0] - ds_densities_ref["density"][:,:,0])
t=4
sns.set_context('talk')
data=(ds_densities_ref_2["density"][:,:,t] - ds_densities_ref["density"][:,:,t]).T
plt.figure(figsize=(20,10))
xlabels = np.full(LONGITUDES, "")
xlabels[30] = "x"
ax = sns.heatmap(data, xticklabels=xlabels, yticklabels=False, cmap=sns.diverging_palette(220, 10, as_cmap=True))
ax.set(title="Diff matrix")
plt.show()

In [ ]:
import netCDF4 as nc
import seaborn as sns
from sim_vars import main_dir_path

ds_parts = nc.Dataset(main_dir_path + "data_double_gyre_ens_30_rad_inf_devinit_1_obscov_100_iterstart_0_iterend_1950_nbobspoint_1_negatcorr_True//parts_ensembles_double_gyre_ens_30_rad_inf_devinit_1_obscov_100_iterstart_0_iterend_1950_nbobspoint_1_negatcorr_True30.nc")

ds_parts["weight"][5,:]

In [ ]:
(ds_parts3['lon'][0,:] - ds_parts2['lon'][0,:]).max()

In [ ]:
ds_parts3.close()

In [ ]:
import seaborn as sns
import numpy as np
import netCDF4 as nc
from sim_vars import ds_densities_ensembles_path, ds_densities_ref_path
from sim_utils import compute_avg_ensembles_densities

t = 0
i = 1900

ds_densities_ensembles = nc.Dataset(ds_densities_ensembles_path, "r")
densities_ensembles = ds_densities_ensembles["density"][:, :, :, t: t + i + 1]
ds_densities_ensembles.close()

avgs_densities = compute_avg_ensembles_densities(densities_ensembles, list(range(t, t+i+1)))[
    :, :, :
]

ds_densities_ref = nc.Dataset(ds_densities_ref_path)
densities_ref = ds_densities_ref["density"][:,:,:]

densities_difference_to_ref = avgs_densities - densities_ref[:, :, t: t + i + 1]
densities_difference_to_ref_abs = np.abs(densities_difference_to_ref)

In [ ]:
import pandas as pd

selectedTimeSteps = [0,1,5,10,50,200,500,1000,1900]

data = pd.DataFrame()
for k in selectedTimeSteps:
    data[k] = densities_difference_to_ref_abs[:,:,k].flatten()

In [ ]:
import matplotlib.pyplot as plt
from sim_vars import end_code_path

sns.set(style="whitegrid")
sns.set_context("talk")
fig = plt.figure(figsize=(16, 10))
ax = sns.boxplot(data=data[selectedTimeSteps], color="0.75", showmeans=True)
ax.set(xlabel="Iteration count \n", ylabel="Absolute density difference")
# ax.set(title="Evolution of density differences between forecast and real situation \n" + end_code_path)

"""
3 situations:
 - Diverges.
 - Converges towards wrong solution.
 - Converges towards right solution.

 METHOD
Equations & Parameters
Model description

One more sim : With negative errors on weights

Put captions before presenting and necessary informations.
Make it clearer in a notebook when it comes to presenting the covariance matrix.

Goal is to help the reader understand in simple terms.
Compromise on what to explain or not. Think about the audience. If it's to be read by a skilled one, redact accordingly.
It's about communicating with the readers.
"""

In [ ]:
from sim_vars import ds_parts_useful_path, NB_ENSEMBLES
import netCDF4 as nc
import numpy as np
import seaborn as sns

ds_parts_ensembles = nc.Dataset(ds_parts_useful_path)
ds_parts_ensembles

## Compute the last std_dev of total mass across ensemble members for a given simulation

TODO

In [ ]:
from sim_vars import ds_parts_ensembles_path, ds_parts_useful_path
import netCDF4 as nc

ds = nc.Dataset(ds_parts_ensembles_path)

ds

### Small code to sort files with numbers in the proper alphabetical order

thingy_10.mp4 => thingy_0010.mp4

thingy_100.mp4 => thingy_0100.mp4

In [ ]:
from sim_vars import output_dir_path
import os
import numpy as np

files = np.array(os.listdir(output_dir_path))
diffrefs = [s for s in files if "from_ref" in s]
diffrefs.sort()
diffrefs_new = diffrefs.copy()
for i in range(len(diffrefs)):
    s: str = diffrefs_new[i]
    end_of_name_index = s.index('.')
    zeros_to_add = 29 - end_of_name_index
    s_new = s[:25] + zeros_to_add * '0' + s[25:]
    os.rename(output_dir_path + s, output_dir_path + s_new)

In [ ]:
from sim_vars import output_dir_path
import os
import numpy as np

files = np.array(os.listdir(output_dir_path))
diffrefs = [s for s in files if "from_origin" in s]
diffrefs.sort()
diffrefs_new = diffrefs.copy()
for i in range(len(diffrefs)):
    s: str = diffrefs_new[i]
    end_of_name_index = s.index('.')
    zeros_to_add = 32 - end_of_name_index
    s_new = s[:28] + zeros_to_add * '0' + s[28:]
    os.rename(output_dir_path + s, output_dir_path + s_new)

In [ ]:
from sim_vars import output_dir_path
import os
import numpy as np

files = np.array(os.listdir("movie_making/"))
diffrefs = [s for s in files if "general" in s]
diffrefs.sort()
diffrefs_new = diffrefs.copy()
for i in range(len(diffrefs)):
    s: str = diffrefs_new[i]
    end_of_name_index = s.index('.')
    zeros_to_add = 12 - end_of_name_index
    s_new = s[:8] + zeros_to_add * '0' + s[8:]
    os.rename("movie_making/" + s, "movie_making/" + s_new)

### Code to see how to animate 4 subplots with heatmaps

In [ ]:
from celluloid import Camera
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

fig, axs = plt.subplots(2,2,figsize=(15,15))
camera = Camera(fig)
sns.set_context("paper")


for t in range(5):
    fig.clear()
    axs = fig.subplots(2,2)
    for i in range(2):
        for j in range(2):
            M = np.array([[1,2],[3,4]])
            axs[i,j] = sns.heatmap(M, ax=axs[i,j])

In [ ]:
import numpy as np